In [1]:

import os
fname_prefix = "/data/users/team2_capstone/code-style-probing/"
os.environ["CUDA_VISIBLE_DEVICES"] = '1'
import numpy as np
import pandas as pd
#import pandas as pd
from transformers import *
import transformers.trainer
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset
from datasets import load_from_disk, load_metric
#import datasets
import pickle
#from utils.helper import read_py150k_code, read_file_to_string
import regex as re
from transformers.data.data_collator import DataCollator, InputDataClass
from torch.utils.data.distributed import DistributedSampler
from torch.utils.data.sampler import RandomSampler
from typing import List, Union, Dict
from torch.utils.data.dataloader import DataLoader
#from transformers.training_args import is_tpu_available
#from transformers.trainer import get_tpu_sampler

In [6]:
tokenizer = RobertaTokenizer.from_pretrained('Salesforce/codet5-small')
tokenizer = RobertaTokenizer.from_pretrained("Salesforce/codet5-base", additional_special_tokens= ["<nl>", "</nl>"])

#train_codet5_dataset = load_from_disk(fname_prefix + 'datasets/combined_model/codet5_train_combined_base.hf')
test_codet5_dataset = load_from_disk(fname_prefix + 'datasets/combined_model_nl_prompt/codet5_test_case_bq_padded.hf')

#debugging_dataset = load_from_disk(fname_prefix + 'datasets/combined_model/codet5_train_comp_bq_padded.hf')

train_comp_dataset = load_from_disk(fname_prefix + 'datasets/combined_model_nl_prompt/codet5_train_comp_bq_padded.hf')
train_casing_dataset = load_from_disk(fname_prefix + 'datasets/combined_model_nl_prompt/codet5_train_case_bq_padded.hf', ).train_test_split(test_size = 0.5)["train"]
train_docstring_dataset = load_from_disk(fname_prefix + 'datasets/combined_model_nl_prompt/codet5_train_docstring_fixed_bq_padded.hf').train_test_split(test_size = 0.9)["train"]
#split for 
train_comment_dataset = load_from_disk(fname_prefix + 'datasets/combined_model_nl_prompt/codet5_train_comment_bq_padded.hf').train_test_split(test_size = 0.5)["train"]
train_class_dataset = load_from_disk(fname_prefix + 'datasets/combined_model_nl_prompt/codet5_train_class_bq_padded.hf').train_test_split(test_size = 0.5)["train"]

loading file https://huggingface.co/Salesforce/codet5-small/resolve/main/vocab.json from cache at /soe/ksmunson/.cache/huggingface/transformers/12dd1d7e87ade1728f8382f3b875a47cdddb88bc50797ecc506957411661a39a.9a48c5abf25554713c6513ab01066e53569b9a2da0d6189715951cf7c6288805
loading file https://huggingface.co/Salesforce/codet5-small/resolve/main/merges.txt from cache at /soe/ksmunson/.cache/huggingface/transformers/c2ada6c76bb6a90c2330323775cb4853dd2b0cfee29d6b2c5ecb419c5874b488.7179059568f1a130b0a79e4bac71f38545207cab0ec45ce82ca09afadb2649a3
loading file https://huggingface.co/Salesforce/codet5-small/resolve/main/added_tokens.json from cache at /soe/ksmunson/.cache/huggingface/transformers/354ae288bdee65437fa8eedecf9a2770001b97bac23d7fc5a04badae8da42346.5cc6e825eb228a7a5cfd27cb4d7151e97a79fb962b31aaf1813aa102e746584b
loading file https://huggingface.co/Salesforce/codet5-small/resolve/main/special_tokens_map.json from cache at /soe/ksmunson/.cache/huggingface/transformers/f420b33feb234c

PermissionError: [Errno 13] Permission denied: '/data/users/team2_capstone/code-style-probing/datasets/combined_model_nl_prompt/codet5_train_case_bq_padded.hf/tmppjrqwfcg'

In [ ]:
from distutils.log import debug


test_codet5_dataset.set_format(type='torch', columns=['input_ids', 'attention_mask', 'labels'])

train_codet5_dataset = {
    #'comp' : debugging_dataset,
    "comp" : train_comp_dataset,
    "case" : train_casing_dataset,
    "class" : train_class_dataset,
    "docstring" : train_docstring_dataset,
    "comment" : train_comment_dataset,
}

for dataset in train_codet5_dataset:
    train_codet5_dataset[dataset].set_format(type='torch', columns=['input_ids', 'attention_mask', 'labels'])

#np.asarray(train_comp_dataset[1]['labels']).dtype
#print (train_comp_dataset[0])

In [ ]:
count = 0
for input in train_codet5_dataset['comp']["input_ids"]:
    if len(input) > 512:
        #print ((input["input_ids"]))
        #idx = (input["input_ids"].index(2))
        #print (idx)
        #print (len(input['attention_mask']))
        break
        count +=1
print (count)

0


In [ ]:
class NLPDataCollator: #(DataCollator):
    """
    Extending the existing DataCollator to work with NLP dataset batches
    """
    def __call__(self, features: List[Union[InputDataClass, Dict]]) -> Dict[str, torch.Tensor]:
    #def collate_batch(self, features: List[Union[InputDataClass, Dict]]) -> Dict[str, torch.Tensor]:
        
        first = features[0]
        #print (first)
        if isinstance(first, dict):
          # NLP data sets current works presents features as lists of dictionary
          # (one per example), so we  will adapt the collate_batch logic for that
          if "labels" in first and first["labels"] is not None:
              if np.asarray(first["labels"]).dtype == torch.int64:
                  labels = [f["labels"] for f in features]
              else:
                  labels = [f["labels"] for f in features]
              batch = {"labels": torch.stack(labels)}
              #print (batch)
          for k, v in first.items():
              if k != "labels" and v is not None and not isinstance(v, str):
                  batch[k] = torch.stack([f[k] for f in features])
          return batch
        else:
          # otherwise, revert to using the default collate_batch
          return DefaultDataCollator().collate_batch(features)


class StrIgnoreDevice(str):
    """
    This is a hack. The Trainer is going call .to(device) on every input
    value, but we need to pass in an additional `task_name` string.
    This prevents it from throwing an error
    """
    def to(self, device):
        return self


class DataLoaderWithTaskname:
    """
    Wrapper around a DataLoader to also yield a task name
    """
    def __init__(self, task_name, data_loader):
        self.task_name = task_name
        self.data_loader = data_loader

        self.batch_size = data_loader.batch_size
        self.dataset = data_loader.dataset

    def __len__(self):
        return len(self.data_loader)
    
    def __iter__(self):
        for batch in self.data_loader:
            batch["task_name"] = StrIgnoreDevice(self.task_name)
            yield batch


class MultitaskDataloader:
    """
    Data loader that combines and samples from multiple single-task
    data loaders.
    """
    def __init__(self, dataloader_dict):
        self.dataloader_dict = dataloader_dict
        self.num_batches_dict = {
            task_name: len(dataloader) 
            for task_name, dataloader in self.dataloader_dict.items()
        }
        self.task_name_list = list(self.dataloader_dict)
        self.dataset = [None] * sum(
            len(dataloader.dataset) 
            for dataloader in self.dataloader_dict.values()
        )

    def __len__(self):
        return sum(self.num_batches_dict.values())

    def __iter__(self):
        """
        For each batch, sample a task, and yield a batch from the respective
        task Dataloader.

        We use size-proportional sampling, but you could easily modify this
        to sample from some-other distribution.
        """
        task_choice_list = []
        for i, task_name in enumerate(self.task_name_list):
            task_choice_list += [i] * self.num_batches_dict[task_name]
        task_choice_list = np.array(task_choice_list)
        np.random.shuffle(task_choice_list)
        dataloader_iter_dict = {
            task_name: iter(dataloader) 
            for task_name, dataloader in self.dataloader_dict.items()
        }
        for task_choice in task_choice_list:
            task_name = self.task_name_list[task_choice]
            yield next(dataloader_iter_dict[task_name])    

class MultitaskTrainer(transformers.Trainer):

    def get_single_train_dataloader(self, task_name, train_dataset):
        """
        Create a single-task data loader that also yields task names
        """
        if self.train_dataset is None:
            raise ValueError("Trainer: training requires a train_dataset.")
        #if is_tpu_available():
        #    train_sampler = get_tpu_sampler(train_dataset)
        #else:
        train_sampler = (
            RandomSampler(train_dataset)
            if self.args.local_rank == -1
            else DistributedSampler(train_dataset)
        )

        data_loader = DataLoaderWithTaskname(
            task_name=task_name,
            data_loader=DataLoader(
              train_dataset,
              batch_size=self.args.train_batch_size,
              sampler=train_sampler,
              collate_fn=self.data_collator.__call__,#collate_batch,
            ),
        )

        #if is_tpu_available():
        #    data_loader = pl.ParallelLoader(
        #        data_loader, [self.args.device]
        #    ).per_device_loader(self.args.device)
        return data_loader

    def get_train_dataloader(self):
        """
        Returns a MultitaskDataloader, which is not actually a Dataloader
        but an iterable that returns a generator that samples from each 
        task Dataloader
        """
        return MultitaskDataloader({
            task_name: self.get_single_train_dataloader(task_name, task_dataset)
            for task_name, task_dataset in self.train_dataset.items()
        })

In [ ]:
class MultitaskModel(transformers.PreTrainedModel):
    def __init__(self, encoder, taskmodels_dict):
        """
        Setting MultitaskModel up as a PretrainedModel allows us
        to take better advantage of Trainer features
        """
        super().__init__(transformers.PretrainedConfig())

        #self.encoder = encoder
        self.taskmodels_dict = nn.ModuleDict(taskmodels_dict)

    @classmethod
    def create(cls, model_name, model_type_dict, model_config_dict):
        """
        #This creates a MultitaskModel using the model class and config objects
        #from single-task models. 

        #We do this by creating each single-task model, and having them share
        #the same encoder transformer.
        """
        shared_encoder = None
        # this could be configured to produce different structured models but for now is one unmodified T5
        taskmodels_dict = {}
        for task_name, model_type in model_type_dict.items():
            model = model_type.from_pretrained(
                model_name, 
                config=model_config_dict[task_name],
            )
            model.resize_token_embeddings(len(tokenizer))

            taskmodels_dict[task_name] = model
        return cls(encoder=shared_encoder, taskmodels_dict=taskmodels_dict)

    def forward(self, task_name, **kwargs):
        return self.taskmodels_dict['model'](**kwargs)

In [ ]:
#model config to factor for task name
#model =AutoModelForSeq2SeqLM.from_pretrained("Salesforce/codet5-small")

model_name = "Salesforce/codet5-small"
multitask_model = MultitaskModel.create(
    model_name=model_name,
    model_type_dict={
        #"stsb": transformers.AutoModelForSequenceClassification,
        #"rte": transformers.AutoModelForSequenceClassification,
        #"commonsense_qa": transformers.AutoModelForMultipleChoice,
        "model" : AutoModelForSeq2SeqLM
    },
    model_config_dict={
        #"stsb": transformers.AutoConfig.from_pretrained(model_name, num_labels=1),
        #"rte": transformers.AutoConfig.from_pretrained(model_name, num_labels=2),
        #"commonsense_qa": transformers.AutoConfig.from_pretrained(model_name),
        "model" : transformers.AutoConfig.from_pretrained(model_name),
    },
)

loading configuration file https://huggingface.co/Salesforce/codet5-small/resolve/main/config.json from cache at /soe/ksmunson/.cache/huggingface/transformers/ef13e715cbf36adda46c74774e8032ab573cfbb2ebe59748c9fc72b7cf67e418.96d28e790b8c3d3e3be663606a66e0793a173c78e745e3603be4c0f878319099
Model config T5Config {
  "_name_or_path": "Salesforce/codet5-small",
  "architectures": [
    "T5ForConditionalGeneration"
  ],
  "bos_token_id": 1,
  "d_ff": 2048,
  "d_kv": 64,
  "d_model": 512,
  "decoder_start_token_id": 0,
  "dense_act_fn": "relu",
  "dropout_rate": 0.1,
  "eos_token_id": 2,
  "feed_forward_proj": "relu",
  "gradient_checkpointing": false,
  "id2label": {
    "0": "LABEL_0"
  },
  "initializer_factor": 1.0,
  "is_encoder_decoder": true,
  "is_gated_act": false,
  "label2id": {
    "LABEL_0": 0
  },
  "layer_norm_epsilon": 1e-06,
  "model_type": "t5",
  "n_positions": 512,
  "num_decoder_layers": 6,
  "num_heads": 8,
  "num_layers": 6,
  "output_past": true,
  "pad_token_id": 0,
 

In [ ]:
batch_size = 24 # switch to 24 once space is available
from transformers import AutoConfig

#combined checkpoint
#model =AutoModelForSeq2SeqLM.from_pretrained(fname_prefix + "/seq2seq_results/combined_full_features_updated_mask_codet5small/checkpoint-97000")


multitask_model.config.max_length = 512
#multitask_model.resize_token_embeddings(len(tokenizer))
args = Seq2SeqTrainingArguments(
    output_dir= fname_prefix + "/seq2seq_results/combined_nl_prompt_base_features_codet5small",
    evaluation_strategy='epoch',
    #eval_steps=20000,
    eval_accumulation_steps=200,
    learning_rate=2e-3,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    weight_decay=0.01,
    save_total_limit=30,
    num_train_epochs=3,
    predict_with_generate=True,
    push_to_hub=False,

)

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).


In [ ]:
print (multitask_model.num_parameters())

60493312


In [ ]:
trainer = MultitaskTrainer(
    multitask_model,
    args,
    train_dataset=train_codet5_dataset,
    eval_dataset=test_codet5_dataset,
    #data_collator=data_collator,
    tokenizer=tokenizer,
    data_collator=NLPDataCollator(),
)

trainer.train()

/data/users/ksmunson/data/users/ksmunson/lib/python3.9/site-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 4081682
  Num Epochs = 3
  Instantaneous batch size per device = 12
  Total train batch size (w. parallel, distributed & accumulation) = 12
  Gradient Accumulation steps = 1
  Total optimization steps = 1020429


Epoch,Training Loss,Validation Loss


KeyboardInterrupt: 

In [ ]:
eval_dataset = test_codet5_dataset
eval_dataset = eval_dataset.train_test_split(test_size = 0.05)
print (eval_dataset)
eval_preds = trainer.predict(eval_dataset['test'])